In [257]:
from abc import ABCMeta, abstractmethod
import numpy as np
from numpy.random import choice
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD as SVD
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.ensemble import RandomTreesEmbedding
#import sythetic_ver2
from sythetic_ver2 import Sythetic
from sythetic_ver2 import txn_class
from sythetic_ver2 import cust_class
from scipy.sparse import issparse
from sklearn.ensemble.bagging import _generate_indices
from sklearn.ensemble.bagging import _generate_bagging_indices
from scipy.sparse import issparse
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [258]:
t1=txn_class(5,[[.9,.1,.0,.0,.0,.0,.0,.0,.0,.0], #single-pattern
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]])

t2=txn_class(5,[[.1,.9,.0,.0,.0,.0,.0,.0,.0,.0], #two-order interactive-items
               [.9,.1,.0,.0,.0,.0,.0,.0,.0,.0],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]])

t3=txn_class(5,[[.1,.1,.1,.1,.1,.1,.1,.1,.1,.1], #two-order interactive-items
               [.1,.0,.0,.0,.1,.0,.1,.0,.7,.0],
               [.1,.0,.0,.0,.1,.1,.1,.6,.0,.0],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
               [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]])

t4=txn_class(5,[[.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],#three-order interactive-items
                [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
                [.0,.0,.0,.0,.1,.8,.1,.0,.0,.0],
                [.0,.0,.0,.0,.0,.1,.8,.1,.0,.0],
                [.0,.0,.0,.0,.0,.0,.1,.8,.1,.0]])

t5=txn_class(5,[[.0,.0,.9,.0,.0,.0,.0,.0,.0,.1],#full-order interactive-items
                [.1,.0,.9,.0,.0,.0,.0,.0,.0,.0],
                [.0,.0,.9,.0,.0,.1,.0,.0,.0,.0],
                [.0,.0,.0,.0,.9,.0,.0,.1,.0,.0],
                [.0,.0,.0,.0,.9,.0,.1,.0,.0,.0]])

t6=txn_class(5,[[.0,.0,.8,.0,.0,.1,.0,.0,.0,.1],#full-order interactive-items
                [.1,.0,.9,.0,.0,.0,.0,.0,.0,.0],
                [.0,.0,.7,.1,.0,.1,.1,.0,.0,.0],
                [.1,.0,.1,.0,.7,.0,.0,.1,.0,.0],
                [.1,.0,.1,.0,.1,.0,.1,.0,.6,.0]])

t7=txn_class(5,[[.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],#random sample
                [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
                [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
                [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1],
                [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]])

c1=cust_class(0,t1,t2,t7)
c2=cust_class(1,t3,t4,t7)
c3=cust_class(2,t5,t6,t7)

s=Sythetic()
s.doc_final([(c1,[50,50,50],50),(c2,[50,50,50],50),(c3,[50,50,50],50)],'full')
s.count2vect()


<150x14267 sparse matrix of type '<class 'numpy.int64'>'
	with 112545 stored elements in Compressed Sparse Row format>

In [259]:
import pandas as pd
pd.DataFrame(s.doc).head()

,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
0,0_0,1_7,2_6,3_6,4_7,0_0_1_7,0_0_2_6,0_0_3_6,0_0_4_7,1_7_2_6,...,0_3_2_5,0_3_3_9,0_3_4_9,1_0_2_5,1_0_3_9,1_0_4_9,2_5_3_9,2_5_4_9,3_9_4_9,3_0_5_9_9
1,0_0,1_5,2_5,3_0,4_7,0_0_1_5,0_0_2_5,0_0_3_0,0_0_4_7,1_5_2_5,...,0_1_2_8,0_1_3_1,0_1_4_5,1_6_2_8,1_6_3_1,1_6_4_5,2_8_3_1,2_8_4_5,3_1_4_5,1_6_8_1_5
2,0_0,1_7,2_2,3_4,4_1,0_0_1_7,0_0_2_2,0_0_3_4,0_0_4_1,1_7_2_2,...,0_2_2_3,0_2_3_4,0_2_4_9,1_3_2_3,1_3_3_4,1_3_4_9,2_3_3_4,2_3_4_9,3_4_4_9,2_3_3_4_9
3,0_0,1_2,2_9,3_7,4_0,0_0_1_2,0_0_2_9,0_0_3_7,0_0_4_0,1_2_2_9,...,0_3_2_5,0_3_3_9,0_3_4_2,1_6_2_5,1_6_3_9,1_6_4_2,2_5_3_9,2_5_4_2,3_9_4_2,3_6_5_9_2
4,0_0,1_6,2_9,3_7,4_1,0_0_1_6,0_0_2_9,0_0_3_7,0_0_4_1,1_6_2_9,...,0_3_2_2,0_3_3_9,0_3_4_6,1_2_2_2,1_2_3_9,1_2_4_6,2_2_3_9,2_2_4_6,3_9_4_6,3_2_2_9_6


In [260]:
class Sampleing(metaclass=ABCMeta):
    @abstractmethod
    def __init__(self,
                 random_state=None, 
                 bootstrap_features=False,
                 bootstrap_samples=False, 
                 n_features=10, 
                 n_samples=10,
                 max_features=5, 
                 max_samples=5):
        self.random_state=random_state
        self.bootstrap_features=bootstrap_features
        self.bootstrap_samples=bootstrap_samples
        self.n_features=n_features
        self.n_samples=n_samples
        self.max_features=max_features
        self.max_samples=max_samples
    
    def fit(self,X):
        if issparse(X):
            self.X=X.toarray()
        else:
            self.X=X
        self.n_samples=len(self.X)
        self.n_features=self.X.shape[1]
        if self.max_samples<=1:
            self.max_samples=int(self.n_samples*self.max_samples)
        if self.max_features<=1:
            self.max_features=int(self.n_features*self.max_features)
        self.feat_indices,self.sample_indices=_generate_bagging_indices(self.random_state, 
                                                              self.bootstrap_features,
                                                              self.bootstrap_samples, 
                                                              self.n_features, 
                                                              self.n_samples,
                                                              self.max_features, 
                                                              self.max_samples)
        
        return self.feat_indices,self.sample_indices
    
    def fit_transform(self,X):
        feat_indices,sample_indices=self.fit(X)
        return (self.X[sample_indices])[:,feat_indices]
    
    def transform(self,new_X):# new_X is a 2D-matrix
        return new_X[:,self.feat_indices]

In [261]:
class Pasting(Sampleing): # Pasting is sampleing samples without replacement
    def __init__(self,
                 random_state=None, 
                 bootstrap_features=False,
                 bootstrap_samples=False, 
                 max_features=1,
                 max_samples=0.8 
                ):

        super().__init__(
                 random_state=random_state, 
                 bootstrap_features=bootstrap_features,
                 bootstrap_samples=bootstrap_samples, 
                 max_features=max_features,
                 max_samples=max_samples 
                 )
        
class Bagging(Sampleing): # Bagging is sampleing samples with replacement(bootstrap)
    def __init__(self,
                 random_state=None, 
                 bootstrap_features=False,
                 bootstrap_samples=True, 
                 max_features=1,
                 max_samples=0.8 
                ):

        super().__init__(
                 random_state=random_state, 
                 bootstrap_features=bootstrap_features,
                 bootstrap_samples=bootstrap_samples, 
                 max_features=max_features,
                 max_samples=max_samples 
                 )
        
class Random_Subspaces(Sampleing): # Random_Subspaces is sampleing features with replacement
    def __init__(self,
                 random_state=None, 
                 bootstrap_features=True,
                 bootstrap_samples=False, 
                 max_features=0.8,
                 max_samples=1
                ):

        super().__init__(
                 random_state=random_state, 
                 bootstrap_features=bootstrap_features,
                 bootstrap_samples=bootstrap_samples, 
                 max_features=max_features,
                 max_samples=max_samples 
                 )
        
class Random_Patches(Sampleing): # Random_Patches is sampleing both on samples and features with replacement
    def __init__(self,
                 random_state=None, 
                 bootstrap_features=True,
                 bootstrap_samples=True, 
                 max_features=0.8,
                 max_samples=0.8
                ):

        super().__init__(
                 random_state=random_state, 
                 bootstrap_features=bootstrap_features,
                 bootstrap_samples=bootstrap_samples, 
                 max_features=max_features,
                 max_samples=max_samples 
                 )

In [282]:
model_dic={'sampling':[Random_Patches],'dim_dedu':[SVD,PCA,KMeans,LDA],'cluster':[KMeans]}
#sampling method available: Pasting,Bagging,Random_Subspaces,
# add more alternatives in 'dim_deduction' and 'cluster'
class estimator():
    
    def __init__(self,model_dic):
        self.layer0=choice(model_dic['sampling'],1)[0]
        self.layer1=choice(model_dic['dim_dedu'],1)[0]
        self.layer2=choice(model_dic['cluster'],1)[0]

        
    def getDimDeduLayer(self,dimMethod,K):
        if dimMethod==PCA or dimMethod==SVD:
            layerToAdd=dimMethod(n_components=K)
        elif dimMethod==KMeans:
            layerToAdd=None
        elif dimMethod==LDA:
            layerToAdd=dimMethod(n_components=K,learning_method='online')
        elif dimMethod==NMF:     
             nmf = dimMethod(n_components=k)
        else:
            print('Error: dimensionality reduction method is not recognized!')
        return layerToAdd 
    
    
    def fit(self,X,K):
        self.layer0_=self.layer0(max_samples=0.9)
        self.layer0_output=self.layer0_.fit_transform(X)
        self.layer1_=self.getDimDeduLayer(self.layer1,K)#self.layer1(n_components=K)
        self.layer1_output=(self.layer0_output if self.layer1_==None else self.layer1_.fit_transform(self.layer0_output))
        self.layer2_=self.layer2(n_clusters=K)
        self.layer2_.fit(self.layer1_output) 
        

    def fit1(self,X,K):
        if self.layer1==KMeans:
            self.pipe=Pipeline([('sampling', self.layer0(max_samples=0.9)), ('cluster', self.layer2(n_clusters=K))])
        else:
            self.pipe=Pipeline([('sampling', self.layer0(max_samples=0.9)), ('dim_dedu', self.getDimDeduLayer(self.layer1,K)),('cluster',  self.layer2(n_clusters=K))])  
        self.pipe.fit(X)
    
    
    def pairwise_predict(self,c1,c2):
        c1_layer0_output=self.layer0_.transform(c1)
        c1_layer1_output=self.layer1_.transform(c1_layer0_output)if self.layer1_!=None else self.layer0_output
        c1_label=self.layer2_.predict(c1_layer1_output)
        c2_layer0_output=self.layer0_.transform(c2)
        c2_layer1_output=self.layer1_.transform(c2_layer0_output)if self.layer1_!=None else self.layer0_output
        c2_label=self.layer2_.predict(c2_layer1_output)
        return (c1_label[0]!=c2_label[0])# if not same class return True
    
    def pairwise_predict1(self,c1,c2):
        c1_label = self.pipe.predict(c1)
        c2_label = self.pipe.predict(c2)
        return (c1_label[0]!=c2_label[0])# if not same class return True



class ensemble():

    def __init__(self,n_estimators,model_dic):
        self.n_setimators=n_estimators
        self.estimators_=[estimator(model_dic) for i in range(self.n_setimators)]
    
    def fit(self,X,K):
        self.training_X=X
        self.numberOfCluster=K
        for est in self.estimators_:
            est.fit(X,K)
    
    def pairwise_predict(self,c1,c2):
        self.est_pred=[]
        for est in self.estimators_:
            self.est_pred.append(est.pairwise_predict(c1,c2))
        prob_diff_class=(sum(self.est_pred)+0.0)/len(self.est_pred)
        return prob_diff_class
    
    def cust_diff_mat(self,X_new):
        new_cust_num=len(X_new)
        cust_diff_mat=np.zeros((new_cust_num,new_cust_num))
        for idx_cust in range(new_cust_num-1):
            for idx_latter in range(idx_cust+1,new_cust_num):
                c1=np.array([X_new[idx_cust]])
                c2=np.array([X_new[idx_latter]])
                cust_diff_mat[idx_cust,idx_latter]=self.pairwise_predict(c1,c2)
        cust_diff_mat+=cust_diff_mat.T  
        #cluster_output=KMeans(n_clusters=self.numberOfCluster).fit(cust_diff_mat).labels_
        return cust_diff_mat
    
    def predict(self,X_new,n_cl):
        return KMeans(n_clusters=n_cl).fit(self.cust_diff_mat(X_new)).labels_

In [283]:
e=ensemble(10,model_dic)

In [284]:
e.fit(X=s.count_mat,K=3)

In [285]:
new_cust=s.count_mat.toarray()[45:55]# from 45-50 customers are class1, 50-55 customers are class2 

In [286]:
e.predict(new_cust,2)
# this is customers mutual difference matrix, 
# the cell with rows_i, col_j represents the prob of customer_i, customer_j are in different class.
# 0. means they are almost the same class, 1. means they are very different that should be different class.
# this value is up to the voting result from ensemble methods, whose voting values are generated by all estimators in ensemble.

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

In [287]:
[e.estimators_[i].layer0 for i in range(5)]

[__main__.Random_Patches,
 __main__.Random_Patches,
 __main__.Random_Patches,
 __main__.Random_Patches,
 __main__.Random_Patches]

In [288]:
[e.estimators_[i].layer1 for i in range(5)]

[sklearn.decomposition.pca.PCA,
 sklearn.decomposition.online_lda.LatentDirichletAllocation,
 sklearn.decomposition.online_lda.LatentDirichletAllocation,
 sklearn.decomposition.pca.PCA,
 sklearn.decomposition.pca.PCA]

In [289]:
[e.estimators_[i].layer2 for i in range(5)]

[sklearn.cluster.k_means_.KMeans,
 sklearn.cluster.k_means_.KMeans,
 sklearn.cluster.k_means_.KMeans,
 sklearn.cluster.k_means_.KMeans,
 sklearn.cluster.k_means_.KMeans]